In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/split-by-person/person_pre1_train.csv
/kaggle/input/split-by-person/person_pre4_test.csv
/kaggle/input/split-by-person/person_pre3_train.csv
/kaggle/input/split-by-person/person_pre2_test.csv
/kaggle/input/split-by-person/person_pre1_test.csv
/kaggle/input/split-by-person/person_pre3_test.csv
/kaggle/input/split-by-person/person_pre2_train.csv
/kaggle/input/split-by-person/person_pre4_train.csv


In [17]:
#!/usr/bin/env python
# coding: utf-8
#get_ipython().run_line_magic('matplotlib', 'widget')

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn import preprocessing, feature_selection, decomposition
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, roc_auc_score, classification_report, make_scorer
from sklearn import tree, ensemble, linear_model, svm
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.combine import SMOTEENN
from sklearn.model_selection import GridSearchCV
import pickle

# from sklearn import metrics
import os

In [18]:
# Select model parameters

#1 2 3
DAY = 2

# red_or_not
# green_or_not
# severity_grade
OUTPUT = 'red_or_not'

# DecisionTree
# RandomForest
# Logistic
# SVM
# all
MODEL = 'SVM'

# SMOTE
# SMOTE_ENN
# NOPE
# weight
IMBAL = 'SMOTE'

In [19]:
# Function for Geting Data
def get_data(train_test, pre_d):
    df = pd.read_csv('../input/split-by-person/' + 'person_pre' + str(pre_d) + '_' + str(train_test) + '.csv', encoding='utf-8')
#     df = pd.read_csv('../input/split-by-time200601/' + 'time_pre' + str(pre_d) + '_' + str(train_test) + '.csv', encoding='utf-8')
#     df = pd.read_csv('../input/ohe-pre/' + str(train_test) + '_OHE_pre' + str(pre_d) + '.csv', encoding='utf-8')
    df.reset_index(drop = True, inplace = True)
    return df

In [20]:
#Prepare Data
train = get_data('train', DAY)
test = get_data('test', DAY)

In [21]:
# Function for Feature Cleaning

def drop_feature(pre_d, df_train):
    col = [ 'ssn',
            'day_record',
            'day_pefr',
            'night_record',
            'night_pefr',
            'date',                           
            'fever',                          
            'night_symptom',                  
            'day_symptom',                    
            'nose_symptom1',                  
            'nose_symptom2',                  
            'nose_symptom3',                 
            'nose_symptom4',                 
            'eye_symptom',                    
            'skin_symptom',                   
            'grade',                         
            'asthma',                         
            'a_rhinitis',                     
            'a_conjunctivitis',               
            'a_dermatitis',                   
            'decode_time',                   
            'SO2',                          
            'CO',                           
            'O3',                           
            'PM10',                        
            'PM2.5',                        
            'NOx',                         
            'PSI',                         
            'avg_humidity',                   
            'avg_tmp',                      
            'max_tmp',                      
            'min_tmp',
            'days',
            'is_OK',                          
            'new_id',
            'pre1_fever',
            'pre1_night_symptom',           
            'pre1_day_symptom',             
            'pre1_nose_symptom1',           
            'pre1_nose_symptom2',           
            'pre1_nose_symptom3',           
            'pre1_nose_symptom4',           
            'pre1_eye_symptom',             
            'pre1_skin_symptom',           
            'pre1_grade',                
            'pre1_asthma',                
            'pre1_a_rhinitis',             
            'pre1_a_conjunctivitis',        
            'pre1_a_dermatitis',          
            'pre1_decode_time',
            'pre1_days',
            'severity_grade_D',
            'severity_grade_N',
           #add:
           #'reference',
            'split_by_time',
            'red_or_not_D',
            'red_or_not_N',
            'green_or_not_D',
            'green_or_not_N',
            'variant'
          ]
    col = col + [str(OUTPUT + '_D'), str(OUTPUT + '_N')]
    if pre_d >= 3:
        col = col + ['pre3_fever',
                     'pre3_night_symptom',           
                     'pre3_day_symptom',             
                     'pre3_nose_symptom1',           
                     'pre3_nose_symptom2',           
                     'pre3_nose_symptom3',           
                     'pre3_nose_symptom4',           
                     'pre3_eye_symptom',             
                     'pre3_skin_symptom',           
                     'pre3_grade',                
                     'pre3_asthma',                
                     'pre3_a_rhinitis',             
                     'pre3_a_conjunctivitis',        
                     'pre3_a_dermatitis',          
                     'pre3_decode_time',
                     'pre3_days'
                    ]
    if pre_d >= 2:
        col = col + ['pre2_fever',
                     'pre2_night_symptom',           
                     'pre2_day_symptom',             
                     'pre2_nose_symptom1',           
                     'pre2_nose_symptom2',           
                     'pre2_nose_symptom3',           
                     'pre2_nose_symptom4',           
                     'pre2_eye_symptom',             
                     'pre2_skin_symptom',           
                     'pre2_grade',                
                     'pre2_asthma',                
                     'pre2_a_rhinitis',             
                     'pre2_a_conjunctivitis',        
                     'pre2_a_dermatitis',          
                     'pre2_decode_time',
                     'pre2_days'
                    ]
        
    feature = df_train.drop(columns = col)
    return feature.copy()

In [22]:
def feature_for_test(pre_d, fe, df_test):
    if pre_d == 1:
        miss_1 = ['pre1_eye_symptom_2.0', 'pre1_a_dermatitis_4.0', 'pre1_asthma_5.0', 'pre1_night_symptom_3.0', 'pre1_skin_symptom_3.0', 'pre1_a_conjunctivitis_3.0']
        fe = fe.drop(columns = miss_1)
        X_test = df_test[list(fe.columns)].copy()
        X_test[miss_1] = 0
    elif pre_d == 2:
#         miss_2 = ['pre2_a_conjunctivitis_3.0', 'pre2_asthma_5.0', 'pre2_skin_symptom_2.0', 'pre1_eye_symptom_2.0', 'pre2_skin_symptom_3.0', 'pre2_a_dermatitis_3.0', 'pre1_a_dermatitis_4.0', 'pre2_a_dermatitis_4.0', 'pre1_asthma_5.0', 'pre2_eye_symptom_2.0', 'pre1_night_symptom_3.0', 'pre2_night_symptom_3.0', 'pre1_skin_symptom_3.0', 'pre1_a_conjunctivitis_3.0']
        miss_2 = []
        fe = fe.drop(columns = miss_2)
        X_test = df_test[list(fe.columns)].copy()
        X_test[miss_2] = 0
    elif pre_d == 3:
        miss_3 = ['pre3_a_dermatitis_4.0', 'pre1_a_dermatitis_4.0', 'pre1_night_symptom_2.0', 'pre2_skin_symptom_3.0', 'pre3_eye_symptom_2.0', 'pre2_a_conjunctivitis_3.0', 'pre3_skin_symptom_3.0', 'pre1_asthma_5.0', 'pre1_night_symptom_3.0', 'pre2_night_symptom_3.0', 'pre1_skin_symptom_3.0', 'pre2_asthma_5.0', 'pre1_eye_symptom_2.0', 'pre3_night_symptom_3.0', 'pre3_a_dermatitis_3.0', 'pre1_a_conjunctivitis_3.0', 'pre2_skin_symptom_2.0', 'pre3_asthma_5.0', 'pre2_a_dermatitis_3.0', 'pre2_a_dermatitis_4.0', 'pre2_eye_symptom_2.0', 'pre3_a_conjunctivitis_3.0', 'pre3_skin_symptom_2.0']
        fe = fe.drop(columns = miss_3)
        X_test = df_test[list(fe.columns)].copy()
        X_test[miss_3] = 0
    return X_test

In [23]:
def sort_feature(fe, X_test):
    X_test = X_test[list(fe.columns)]
    return X_test.copy()

In [24]:
#get feature
feature = drop_feature(DAY, train)

#prepare X,y for training
X_D = train[list(feature.columns)].copy()
y_D = train[str(OUTPUT + '_D')].copy()

In [25]:
feature_select = ['pre2_night_record', 'pre1_avg_tmp',  'pre2_day_record', 'pre2_min_tmp',
 'pre1_min_tmp', 'pre1_night_pefr', 'pre1_max_tmp', 'pre1_PM2.5', 'pre1_SO2',
 'pre2_nose_symptom1_0.0', 'pre2_a_rhinitis_2.0', 'pre2_SO2',
 'pre2_day_symptom_1.0', 'pre2_nose_symptom4_0.0', 'pre2_asthma_1.0',
 'pre2_NOx', 'pre2_PSI', 'pre1_nose_symptom1_0.0', 'pre1_grade_1.0',
 'pre1_night_record', 'pre1_nose_symptom2_1.0', 'pre1_nose_symptom4_1.0',
 'pre1_day_symptom_1.0', 'pre2_nose_symptom2_0.0', 'pre1_a_rhinitis_2.0',
 'pre1_day_pefr', 'pre1_CO', 'pre1_skin_symptom_0.0', 'pre1_a_dermatitis_1.0',
 'pre1_PM10', 'pre2_eye_symptom_0.0', 'pre2_a_conjunctivitis_1.0',
 'pre1_eye_symptom_0.0', 'pre1_a_conjunctivitis_1.0',
 'pre2_skin_symptom_0.0', 'pre2_a_dermatitis_1.0', 'pre1_avg_humidity',
 'pre1_tmp_diff', 'pre2_O3', 'pre2_night_pefr', 'pre2_variant', 'pre1_O3',
 'pre2_max_tmp', 'pre2_PM2.5', 'pre2_avg_humidity', 'pre1_day_record',
 'pre1_PSI', 'pre1_variant', 'reference', 'pre2_tmp_diff']

In [26]:
#feature engineering
X_D = X_D[list(feature_select)].copy()

In [27]:
#handle Imbalance
if IMBAL == 'SMOTE':
    sm_D = SMOTE(random_state=42)
    X_D, y_D = sm_D.fit_resample(X_D, y_D)

elif IMBAL == 'SMOTE_ENN':
    sme_D = SMOTEENN(random_state=42)
    X_D, y_D = sme_D.fit_resample(X_D, y_D)
    
elif IMBAL == 'weight':
    print('Do weight adjustment')
else:
    print('Did not handle imbalance')

In [28]:
#get test data features:X
X_test = feature_for_test(DAY, feature, test)
X_test = sort_feature(feature, X_test)
X_test = X_test[list(feature_select)].copy()

#get test data output:y
y_test_D = test[str(OUTPUT + '_D')].copy()

In [29]:
if MODEL == 'SVM':
    
    clf = svm.SVC(kernel = 'rbf',
            random_state = 42,
            probability = True,
            #tuned
            C = 0.05,
            gamma = 'scale'
           )
    #'C': [0.8], 'gamma': ['scale']
    #
        
    clf.fit(X_D, y_D)
    # pickle model
    filename = 'trained_asthma_model.pkl'
    pickle.dump(clf, open(filename, 'wb'))
 
    # some time later...
 
    # load the model from disk
#     loaded_model = pickle.load(open(filename, 'rb'))
    
    
    y_true, y_pred = y_test_D, clf.predict(X_test)
    print(clf)
    print(classification_report(y_true, y_pred, digits=3))
    print()
elif MODEL == 'Logistic':
    clf = linear_model.LogisticRegression(random_state=42,
                                    #tuned
                                    C = 0.01
                                    )
    
    for rfe_num in range(20, 21, 10):
        for pca_num in range(20, rfe_num + 1, 5):
            rfe = feature_selection.RFE(clf, n_features_to_select = rfe_num)
            pc = decomposition.PCA(n_components = pca_num)
            
            #
            fit = rfe.fit(X_D, y_D)
            rX_D = fit.transform(X_D) 
            rX_test = fit.transform(X_test)
            #
            fit = pc.fit(rX_D)
            pX_D = fit.transform(rX_D) 
            pX_test = fit.transform(rX_test)
            
            clf.fit(pX_D, y_D)
            y_true, y_pred = y_test_D, clf.predict(pX_test)
            print('feature_num:', rfe_num, pca_num)
            print(clf)
            print(classification_report(y_true, y_pred, digits=3))
            print()

else:
    print('No such model!!!')

SVC(C=0.05, probability=True, random_state=42)
              precision    recall  f1-score   support

           0      0.993     0.856     0.919       660
           1      0.144     0.800     0.244        20

    accuracy                          0.854       680
   macro avg      0.569     0.828     0.582       680
weighted avg      0.968     0.854     0.900       680




In [30]:
# def plot_coefficients(classifier, feature_names, top_features=10):
#     coef = classifier.coef_.ravel()
#     top_positive_coefficients = np.argsort(coef)[-top_features:]
#     top_negative_coefficients = np.argsort(coef)[:top_features]
#     top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
#     # create plot
#     plt.figure(figsize=(15, 5))
#     colors = ['red' if c < 0 else 'blue' for c in coef[top_coefficients]]
#     plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
#     feature_names = np.array(feature_names)
#     plt.xticks(np.arange(1, 1 + 2 * top_features), feature_names[top_coefficients], rotation=60, ha='right')
#     (print(feature_names[top_coefficients]))
#     plt.show()

In [31]:
# plot_coefficients(clf, X_D.columns)